In [2]:
import collections
import logging
import os
from typing import Optional, Tuple
#from tqdm.auto import tqdm

# misc
import os
import re
import time
import ast
import warnings
import math
import copy
import matplotlib.pyplot as plt
# from xgboost import plot_importance
# import seaborn as sns

# data
import pandas as pd
import numpy as np
import csv
from torch.utils.data import random_split
import datasets

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn import metrics

#import scikitplot as skplt
# import xgboost
# import tensorflow as tf
import torch

from transformers import DefaultDataCollator, EvalPrediction
from transformers import AutoTokenizer, AutoModelForSequenceClassification, XLMRobertaForSequenceClassification
from transformers import DistilBertConfig, DistilBertModel, BertTokenizerFast
from transformers import AutoModel
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModelForQuestionAnswering, DistilBertForSequenceClassification, DistilBertModel, CamembertForSequenceClassification, RobertaForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import json
from transformers import TrainerCallback, EarlyStoppingCallback
from sklearn.metrics import accuracy_score
from collections import Counter
import torch
import evaluate
import optuna

class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")

### Compute_metrics function for Question and Answering problem is different to classification, more preocessing required.

metric = evaluate.load("squad_v2")

def compute_metrics(p: EvalPrediction):
        return metric.compute(predictions=p.predictions, references=p.label_ids)

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

c:\Users\Alex\miniconda3\envs\pytorch-modified\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU: NVIDIA GeForce RTX 4060 is available.
cuda


In [7]:
"""
A subclass of `Trainer` specific to Question-Answering tasks
"""
import math
import time

from transformers import Trainer
# , is_torch_xla_available
from transformers.trainer_utils import PredictionOutput, speed_metrics


# if is_torch_xla_available():
#     import torch_xla.core.xla_model as xm
#     import torch_xla.debug.metrics as met


class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, post_process_function=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples
        self.post_process_function = post_process_function

    def evaluate(self, eval_dataset=None, eval_examples=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                eval_dataloader,
                description="Evaluation",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )
        if self.post_process_function is not None and self.compute_metrics is not None and self.args.should_save:
            # Only the main node write the results by default
            eval_preds = self.post_process_function(eval_examples, eval_dataset, output.predictions)
            metrics = self.compute_metrics(eval_preds)

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
            metrics.update(output.metrics)
        else:
            metrics = output.metrics

        if self.args.should_log:
            # Only the main node log the results by default
            self.log(metrics)

        if self.args.tpu_metrics_debug or self.args.debug:
            # tpu-comment: Logging debug metrics for PyTorch/XLA (compile, execute times, ops, etc.)
            xm.master_print(met.metrics_report())

        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        return metrics

    def predict(self, predict_dataset, predict_examples, ignore_keys=None, metric_key_prefix: str = "test"):
        predict_dataloader = self.get_test_dataloader(predict_dataset)

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                predict_dataloader,
                description="Prediction",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )

        if self.post_process_function is None or self.compute_metrics is None:
            return output

        predictions = self.post_process_function(predict_examples, predict_dataset, output.predictions, "predict")
        metrics = self.compute_metrics(predictions)

        # Prefix all keys with metric_key_prefix + '_'
        for key in list(metrics.keys()):
            if not key.startswith(f"{metric_key_prefix}_"):
                metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
        metrics.update(output.metrics)
        return PredictionOutput(predictions=predictions.predictions, label_ids=predictions.label_ids, metrics=metrics)

In [3]:
max_length = 512
stride = 128
model = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model)

## The dataset does not have the ending index of the "Answers" which is required for the Question and Answering transformer.
## This function uses the data in the Squad dataset to extract the start index and end index as well as tokenizes the dataset.
## after running this function, the data is ready to be used in the model

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation=True,
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        if len(answer["answer_start"]) > 0:
            
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
        else:
            start_char = 0
            end_char = 0
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs



c:\Users\Alex\miniconda3\envs\pytorch-modified\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def prepare_validation_features(examples):
  # Some of the questions have lots of whitespace on the left, which is not useful and will make the
  # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
  # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]
  # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
  # in one example possible giving several features when a context is long, each of those features having a
  # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
      examples["question"],
      examples["context"],
      truncation= True,
      # max_length=max_length,
      # stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

  # Since one example might give us several features if it has a long context, we need a map from a feature to
  # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

  # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
  # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
      # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 0

      # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

      # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
      # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
      ]

    return tokenized_examples

In [9]:
logger = logging.getLogger(__name__)

def postprocess_qa_predictions(
    examples,
    features,
    predictions: Tuple[np.ndarray, np.ndarray],
    version_2_with_negative: bool = True,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0,
    output_dir: Optional[str] = None,
    prefix: Optional[str] = None,
    log_level: Optional[int] = logging.WARNING,
):
    """
    Post-processes the predictions of a question-answering model to convert them to answers that are substrings of the
    original contexts. This is the base postprocessing functions for models that only return start and end logits.

    Args:
        examples: The non-preprocessed dataset (see the main script for more information).
        features: The processed dataset (see the main script for more information).
        predictions (:obj:`Tuple[np.ndarray, np.ndarray]`):
            The predictions of the model: two arrays containing the start logits and the end logits respectively. Its
            first dimension must match the number of elements of :obj:`features`.
        version_2_with_negative (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether or not the underlying dataset contains examples with no answers.
        n_best_size (:obj:`int`, `optional`, defaults to 20):
            The total number of n-best predictions to generate when looking for an answer.
        max_answer_length (:obj:`int`, `optional`, defaults to 30):
            The maximum length of an answer that can be generated. This is needed because the start and end predictions
            are not conditioned on one another.
        null_score_diff_threshold (:obj:`float`, `optional`, defaults to 0):
            The threshold used to select the null answer: if the best answer has a score that is less than the score of
            the null answer minus this threshold, the null answer is selected for this example (note that the score of
            the null answer for an example giving several features is the minimum of the scores for the null answer on
            each feature: all features must be aligned on the fact they `want` to predict a null answer).

            Only useful when :obj:`version_2_with_negative` is :obj:`True`.
        output_dir (:obj:`str`, `optional`):
            If provided, the dictionaries of predictions, n_best predictions (with their scores and logits) and, if
            :obj:`version_2_with_negative=True`, the dictionary of the scores differences between best and null
            answers, are saved in `output_dir`.
        prefix (:obj:`str`, `optional`):
            If provided, the dictionaries mentioned above are saved with `prefix` added to their names.
        log_level (:obj:`int`, `optional`, defaults to ``logging.WARNING``):
            ``logging`` log level (e.g., ``logging.WARNING``)
    """
    if len(predictions) != 2:
        raise ValueError("`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    if version_2_with_negative:
        scores_diff_json = collections.OrderedDict()

    # Logging.
    logger.setLevel(log_level)
    logger.info(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or len(offset_mapping[start_index]) < 2
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[end_index]) < 2
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative and min_null_prediction is not None:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if (
            version_2_with_negative
            and min_null_prediction is not None
            and not any(p["offsets"] == (0, 0) for p in predictions)
        ):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            scores_diff_json[example["id"]] = float(score_diff)  # To be JSON-serializable.
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        # Make `predictions` JSON-serializable by casting np.float back to float.
        all_nbest_json[example["id"]] = [
            {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
            for pred in predictions
        ]

    # If we have an output_dir, let's save all those dicts.
    if output_dir is not None:
        if not os.path.isdir(output_dir):
            raise EnvironmentError(f"{output_dir} is not a directory.")

        prediction_file = os.path.join(
            output_dir, "predictions.json" if prefix is None else f"{prefix}_predictions.json"
        )
        nbest_file = os.path.join(
            output_dir, "nbest_predictions.json" if prefix is None else f"{prefix}_nbest_predictions.json"
        )
        if version_2_with_negative:
            null_odds_file = os.path.join(
                output_dir, "null_odds.json" if prefix is None else f"{prefix}_null_odds.json"
            )

        logger.info(f"Saving predictions to {prediction_file}.")
        with open(prediction_file, "w") as writer:
            writer.write(json.dumps(all_predictions, indent=4) + "\n")
        logger.info(f"Saving nbest_preds to {nbest_file}.")
        with open(nbest_file, "w") as writer:
            writer.write(json.dumps(all_nbest_json, indent=4) + "\n")
        if version_2_with_negative:
            logger.info(f"Saving null_odds to {null_odds_file}.")
            with open(null_odds_file, "w") as writer:
                writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

    return all_predictions

In [11]:
def post_processing_function(examples, features, predictions, stage="eval"):
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions = postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            max_answer_length=max_length
        )
        # Format the result to the format the metric expects.
        if 1==1:
            formatted_predictions = [
                {"id": str(k), "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
        else:
            formatted_predictions = [{"id": str(k), "prediction_text": v} for k, v in predictions.items()]

        references = [{"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples]
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)

In [5]:
## IMPORT RAW DATASET

squad_raw = load_dataset("squad_v2", split="train[:3500]+validation[:20]")
train_testvalid = squad_raw.train_test_split(test_size=0.2, shuffle=True)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, shuffle=True)

squad_raw = datasets.DatasetDict({
                                'train': train_testvalid['train'],
                                'validation': test_valid['train'],
                                'test': test_valid['test']
                                })

In [12]:
squad_raw

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2816
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 352
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 352
    })
})

In [6]:
## APPLY PREPROCESSING
train_dataset = squad_raw['train'].map(
                preprocess_function,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
            )
eval_dataset = squad_raw['validation'].map(
                prepare_validation_features,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on validation dataset",
            )
eval_examples =  squad_raw["validation"]


Running tokenizer on validation dataset: 100%|██████████| 352/352 [00:00<00:00, 2957.30 examples/s]


In [9]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 2826
})

In [19]:
eval_examples["answers"]

[{'text': ['Naomie Harris'], 'answer_start': [166]},
 {'text': [], 'answer_start': []},
 {'text': ['Peter Minuit'], 'answer_start': [530]},
 {'text': ['the four-bar phrase'], 'answer_start': [879]},
 {'text': ['his father'], 'answer_start': [1166]},
 {'text': ['5 GB'], 'answer_start': [1369]},
 {'text': ['very destructive'], 'answer_start': [185]},
 {'text': ['third generation'], 'answer_start': [4]},
 {'text': ['Vogue'], 'answer_start': [364]},
 {'text': ['Dangerously in Love'], 'answer_start': [505]},
 {'text': ['pop, soul and funk'], 'answer_start': [60]},
 {'text': ['almost 80%'], 'answer_start': [341]},
 {'text': ['20'], 'answer_start': [16]},
 {'text': ['Liszt'], 'answer_start': [243]},
 {'text': ['Romantic era'], 'answer_start': [276]},
 {'text': ['hearing in them all the most beautiful things of art'],
  'answer_start': [378]},
 {'text': ['Daniel Craig'], 'answer_start': [93]},
 {'text': ['Thunderball.'], 'answer_start': [326]},
 {'text': ['Michael Jackson'], 'answer_start': [1

In [ ]:
###############################################################
# UNCOMMENT THIS TO TEST TRAINER BEFORE STARTING TUNING
###############################################################

data_collator = DefaultDataCollator()
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
training_args = TrainingArguments(
    output_dir="test",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=eval_examples,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,)

trainer.train()

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/352 [00:00<?, ?it/s]

{'eval_exact': 0.8522727272727273, 'eval_f1': 0.8522727272727273, 'eval_total': 352, 'eval_HasAns_exact': 0.0, 'eval_HasAns_f1': 0.0, 'eval_HasAns_total': 313, 'eval_NoAns_exact': 7.6923076923076925, 'eval_NoAns_f1': 7.6923076923076925, 'eval_NoAns_total': 39, 'eval_best_exact': 11.079545454545455, 'eval_best_exact_thresh': 0.0, 'eval_best_f1': 11.079545454545455, 'eval_best_f1_thresh': 0.0, 'eval_runtime': 8.1201, 'eval_samples_per_second': 43.349, 'eval_steps_per_second': 1.355, 'epoch': 1.0}


...

{'eval_exact': 0.0, 'eval_f1': 0.0, 'eval_total': 352, 'eval_HasAns_exact': 0.0, 'eval_HasAns_f1': 0.0, 'eval_HasAns_total': 313, 'eval_NoAns_exact': 0.0, 'eval_NoAns_f1': 0.0, 'eval_NoAns_total': 39, 'eval_best_exact': 11.079545454545455, 'eval_best_exact_thresh': 0.0, 'eval_best_f1': 11.079545454545455, 'eval_best_f1_thresh': 0.0, 'eval_runtime': 7.2538, 'eval_samples_per_second': 48.526, 'eval_steps_per_second': 1.516, 'epoch': 2.0}


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/352 [00:00<?, ?it/s]

{'eval_exact': 0.0, 'eval_f1': 0.0, 'eval_total': 352, 'eval_HasAns_exact': 0.0, 'eval_HasAns_f1': 0.0, 'eval_HasAns_total': 313, 'eval_NoAns_exact': 0.0, 'eval_NoAns_f1': 0.0, 'eval_NoAns_total': 39, 'eval_best_exact': 11.079545454545455, 'eval_best_exact_thresh': 0.0, 'eval_best_f1': 11.079545454545455, 'eval_best_f1_thresh': 0.0, 'eval_runtime': 5.8478, 'eval_samples_per_second': 60.193, 'eval_steps_per_second': 1.881, 'epoch': 3.0}
{'train_runtime': 5098.4602, 'train_samples_per_second': 1.664, 'train_steps_per_second': 0.052, 'train_loss': 2.0422566731770835, 'epoch': 3.0}


TrainOutput(global_step=267, training_loss=2.0422566731770835, metrics={'train_runtime': 5098.4602, 'train_samples_per_second': 1.664, 'train_steps_per_second': 0.052, 'total_flos': 2216841684148224.0, 'train_loss': 2.0422566731770835, 'epoch': 3.0})

In [ ]:
torch.cuda.empty_cache()

In [ ]:

### DATA FOR HYPERPARAMETER TUNING

squad_raw = load_dataset("squad", split="train[:5000]+validation[:2000]")
train_testvalid = squad_raw.train_test_split(test_size=0.2, shuffle=True)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, shuffle=True)

squad_raw = datasets.DatasetDict({
                                'train': train_testvalid['train'],
                                'validation': test_valid['train'],
                                'test': test_valid['test']
                                })
train_dataset = squad_raw['train'].map(
                preprocess_function,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
            )
eval_dataset = squad_raw['validation'].map(
                prepare_validation_features,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on validation dataset",
            )
eval_examples =  squad_raw["validation"]

Running tokenizer on validation dataset:   0%|          | 0/700 [00:00<?, ? examples/s]

In [ ]:

# Define model initialization function, "model" parameter is defined in the second cell of the notebook
def model_init():
    return AutoModelForQuestionAnswering.from_pretrained(model)

# Hyperparameter tuning with Optuna
def objective(trial):
    data_collator = DefaultDataCollator()
    # Hyperparameters to tune
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    warmup_steps = trial.suggest_int('warmup_steps', 0, 500)
    weight_decay = trial.suggest_float('weight_decay', 0.01, 0.3)
    # Training arguments
    arguments = TrainingArguments(
        output_dir="testOutput",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=10,  # More epochs
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        warmup_steps=warmup_steps,
        weight_decay=weight_decay
    )
    trainer = QuestionAnsweringTrainer(
        model=model,
        args=arguments,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=eval_examples,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    # Evaluate the model
    eval_results = trainer.evaluate()
    return eval_results['f1']

In [ ]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=20)

In [ ]:
# print("Best Trial:")
# print("Value (Performance Metric):", study.best_trial.value)
# print("Parameters:")
# for key, value in study.best_trial.params.items():
#     print(f"{key}: {value}")